In [ ]:
import pyvisa

rm = pyvisa.ResourceManager()
print(rm)
Tmon8 = rm.open_resource('ASRL5::INSTR')
Tmon8.baud_rate = 115200
Tmon8.read_termination = '\r\n'
Tmon8.write_termination = '\r\n'
id = Tmon8.query('*IDN?')
print (id)

command_bytes = b'KRDG\xa3\xbf1'
Tmon8.write_raw(command_bytes)
temperature = Tmon8.read()
print(temperature)

In [ ]:
import serial
from time import sleep
ser = serial.Serial("COM4", 115200)
for i in range(10):
    ser.write(f"Trim:{i}".encode('ascii'))
    Trim = ser.readline()
    print(f'Received: {Trim.decode()}')
    sleep(0.5)

In [ ]:
import serial
 
# 打开 COM17，将波特率配置为115200.
ser = serial.Serial(port="COM4", baudrate=115200)
 
# 串口发送 ABCDEFG，并输出发送的字节数。
write_len = ser.write("Trim:2001".encode('ascii'))
print("串口发出{}个字节。".format(write_len))
 
ser.close()

In [ ]:
import serial
import time

A = True
try:
    # 确保所有参数（特别是timeout）都已设置
    ser = serial.Serial(
        port="COM4",
        baudrate=115200,
        timeout=0.2  # 设置一个较短的超时，比如0.2秒，用于判断数据流是否结束
    )
    print(f"串口 {ser.name} 已打开。")

    while A:
        # 1. 发送指令，并加上换行符，这几乎是必须的
        command = "Trim:458\r\n"
        ser.write(command.encode('ascii'))
        print(f"[Tx] 发送指令: {command.strip()}")

        # 2. 等待设备开始响应
        # 从日志看，至少有10ms延迟，我们多等一会儿
        time.sleep(0.05) 

        # 3. 循环读取所有返回的行
        print("[Rx] 开始接收数据...")
        response_lines = []
        empty_read_count = 0
        max_empty_reads = 3

        while True:
            # 读取一行数据
            line = ser.readline()

            # 如果读到了数据
            if line:
                # 将字节串解码为字符串，并去掉两端的空白（包括\r\n）
                decoded_line = line.decode('ascii', errors='ignore').strip()
                print(decoded_line)  # 实时打印接收到的每一行
                response_lines.append(decoded_line)

                # 检查是否是结束标志
                if "================================" in decoded_line:
                    print("[Rx] 接收到结束标志，数据接收完毕。")
                    break
            # 如果没读到数据（readline超时返回空字节串）
            else:
                empty_read_count += 1
                # print(f"[Rx] 读取超时，第 {empty_read_count} 次...")
                
                if empty_read_count >= max_empty_reads:
                    print(f"[Rx] 连续 {max_empty_reads} 次读取超时，认为数据已全部接收。")
                    break
                else:
                    continue
        
        # 打印完整的接收内容（可选）
        print("\n--- 完整响应内容 ---")
        for l in response_lines:
            print(l)
        print("---------------------\n")

        # 4. 等待一段时间再进行下一次循环，避免过于频繁地发送
        time.sleep(2)
        A = False

except serial.SerialException as e:
    print(f"打开串口时出错: {e}")

finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()
        print(f"串口 {ser.name} 已关闭。")

串口 COM4 已打开。
[Tx] 发送指令: Trim:458
[Rx] 开始接收数据...
mode:Trim
setting:458

[Rx] 读取超时，第 1 次...
[Rx] 读取超时，第 2 次...

=== New Setting Received ===
Mode: Trim
Setting string: '458'
16-bit value: 458 (0x01CA)
Binary: 0000 0001 1100 1010

Bit Allocation:
LDO3   (Bit 15): 0
LDO2   (Bit 14): 0
LDO1   (Bit 13): 0
LDO0   (Bit 12): 0
R1_IN3 (Bit 11): 0
R1_IN2 (Bit 10): 0
R1_IN1 (Bit 9):  0
R1_IN0 (Bit 8):  1
R2_IN3 (Bit 7):  1
R2_IN2 (Bit 6):  1
R2_IN1 (Bit 5):  0
R2_IN0 (Bit 4):  0
R3_IN3 (Bit 3):  1
R3_IN2 (Bit 2):  0
R3_IN1 (Bit 1):  1
R3_IN0 (Bit 0):  0
[Rx] 接收到结束标志，数据接收完毕。

--- 完整响应内容 ---
mode:Trim
setting:458


=== New Setting Received ===
Mode: Trim
Setting string: '458'
16-bit value: 458 (0x01CA)
Binary: 0000 0001 1100 1010

Bit Allocation:
LDO3   (Bit 15): 0
LDO2   (Bit 14): 0
LDO1   (Bit 13): 0
LDO0   (Bit 12): 0
R1_IN3 (Bit 11): 0
R1_IN2 (Bit 10): 0
R1_IN1 (Bit 9):  0
R1_IN0 (Bit 8):  1
R2_IN3 (Bit 7):  1
R2_IN2 (Bit 6):  1
R2_IN1 (Bit 5):  0
R2_IN0 (Bit 4):  0
R3_IN3 (Bit 3):  1
R3_IN2 (Bi